# Multimodal Model with Fast.ai

Combining images + tabular features using late fusion architecture.

**Key Innovation**: Use auxiliary learning during training to predict NDVI/Height from images.
This helps the model learn visual features that correlate with these metrics, even though they won't be available at test time.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# Fast.ai imports
from fastai.vision.all import *
from fastai.tabular.all import *
from fastai.callback.all import *

# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler

sns.set_style('whitegrid')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Set seeds
np.random.seed(42)
torch.manual_seed(42)
set_seed(42, reproducible=True)

## 1. Load and Prepare Data

In [ ]:
# Load training data
train_df = pd.read_csv('competition/train.csv')

# Convert to wide format
train_wide = train_df.pivot_table(
    index=['image_path', 'Sampling_Date', 'State', 'Species', 'Pre_GSHH_NDVI', 'Height_Ave_cm'],
    columns='target_name',
    values='target'
).reset_index()

print(f"Total images: {len(train_wide)}")
train_wide.head()

In [ ]:
# Define target columns
target_cols = ['Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g']

# Add full image paths
train_wide['full_image_path'] = train_wide['image_path'].apply(lambda x: f'competition/{x}')

# Extract date features
train_wide['Sampling_Date'] = pd.to_datetime(train_wide['Sampling_Date'])
train_wide['Month'] = train_wide['Sampling_Date'].dt.month
train_wide['DayOfYear'] = train_wide['Sampling_Date'].dt.dayofyear
train_wide['Year'] = train_wide['Sampling_Date'].dt.year

# Define tabular features
tabular_features = ['Pre_GSHH_NDVI', 'Height_Ave_cm', 'Month', 'DayOfYear', 'State', 'Species']

# Split train/validation
train_data, val_data = train_test_split(train_wide, test_size=0.2, random_state=42)

print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")
print(f"\nTabular features: {tabular_features}")
print(f"Target columns: {target_cols}")

## 2. Create Multimodal Dataset

Custom dataset that loads both images and tabular features.

In [ ]:
class MultimodalPastureDataset(Dataset):
    def __init__(self, dataframe, tabular_features, target_cols, 
                 cat_features=['State', 'Species'], 
                 cont_features=['Pre_GSHH_NDVI', 'Height_Ave_cm', 'Month', 'DayOfYear'],
                 image_size=224, augment=False):
        """
        Dataset that combines images and tabular data
        """
        self.df = dataframe.reset_index(drop=True)
        self.tabular_features = tabular_features
        self.target_cols = target_cols
        self.cat_features = cat_features
        self.cont_features = cont_features
        self.image_size = image_size
        self.augment = augment
        
        # Encode categorical variables
        self.cat_encoders = {}
        for col in cat_features:
            unique_vals = self.df[col].unique()
            self.cat_encoders[col] = {val: i for i, val in enumerate(unique_vals)}
        
        # Scale continuous features
        self.scaler = StandardScaler()
        self.df[cont_features] = self.scaler.fit_transform(self.df[cont_features])
        
        # Image transforms
        if augment:
            self.transforms = transforms.Compose([
                transforms.Resize((image_size, image_size)),
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.RandomRotation(15),
                transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        else:
            self.transforms = transforms.Compose([
                transforms.Resize((image_size, image_size)),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # Load and transform image
        img_path = row['full_image_path']
        image = Image.open(img_path).convert('RGB')
        image = self.transforms(image)
        
        # Get categorical features
        cat_data = torch.tensor(
            [self.cat_encoders[col][row[col]] for col in self.cat_features],
            dtype=torch.long
        )
        
        # Get continuous features
        cont_data = torch.tensor(
            row[self.cont_features].values.astype('float32'),
            dtype=torch.float32
        )
        
        # Get targets
        targets = torch.tensor(
            row[self.target_cols].values.astype('float32'),
            dtype=torch.float32
        )
        
        # Also return NDVI and Height for auxiliary learning
        # Note: These are the ORIGINAL unscaled values from before scaling
        ndvi_height = torch.tensor(
            [row['Pre_GSHH_NDVI'], row['Height_Ave_cm']],
            dtype=torch.float32
        )
        
        return image, cat_data, cont_data, targets, ndvi_height

# Create datasets
train_dataset = MultimodalPastureDataset(
    train_data.copy(),
    tabular_features,
    target_cols,
    augment=True
)

val_dataset = MultimodalPastureDataset(
    val_data.copy(),
    tabular_features,
    target_cols,
    augment=False
)

# Create dataloaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

print(f"Train batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")

In [ ]:
# Test the dataset
sample = train_dataset[0]
image, cat_data, cont_data, targets, ndvi_height = sample

print(f"Image shape: {image.shape}")
print(f"Categorical data shape: {cat_data.shape}")
print(f"Continuous data shape: {cont_data.shape}")
print(f"Targets shape: {targets.shape}")
print(f"NDVI/Height shape: {ndvi_height.shape}")
print(f"\nTargets: {targets}")

## 3. Build Multimodal Model with Auxiliary Learning

Architecture:
- **Image branch**: ResNet50 → image features
- **Tabular branch**: Embedding + Dense → tabular features  
- **Fusion**: Concatenate both → Dense layers → predictions
- **Auxiliary**: Image features → predict NDVI/Height (training only)

In [ ]:
class MultimodalBiomassPredictor(nn.Module):
    def __init__(self, 
                 num_cat_features=2,
                 cat_embedding_sizes=[10, 20],  # State, Species
                 num_cont_features=4,
                 num_outputs=5,
                 image_model='resnet50',
                 use_auxiliary=True):
        super().__init__()
        
        self.use_auxiliary = use_auxiliary
        
        # ========== IMAGE BRANCH ==========
        # Load pre-trained ResNet50
        if image_model == 'resnet50':
            self.image_encoder = models.resnet50(pretrained=True)
            image_features = 2048
        elif image_model == 'resnet34':
            self.image_encoder = models.resnet34(pretrained=True)
            image_features = 512
        
        # Remove the final FC layer
        self.image_encoder = nn.Sequential(*list(self.image_encoder.children())[:-1])
        
        # ========== TABULAR BRANCH ==========
        # Embeddings for categorical variables
        self.cat_embeddings = nn.ModuleList([
            nn.Embedding(size, emb_size)
            for size, emb_size in zip([len(train_dataset.cat_encoders[f]) for f in ['State', 'Species']], 
                                       cat_embedding_sizes)
        ])
        
        # Total tabular features size
        tabular_size = sum(cat_embedding_sizes) + num_cont_features
        
        # Tabular processing
        self.tabular_encoder = nn.Sequential(
            nn.Linear(tabular_size, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        
        # ========== FUSION ==========
        # Combine image + tabular features
        combined_size = image_features + 32
        
        self.fusion_head = nn.Sequential(
            nn.Linear(combined_size, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.4),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.3),
            nn.Linear(256, num_outputs)
        )
        
        # ========== AUXILIARY HEAD ==========
        # Predict NDVI and Height from image features (training only)
        if use_auxiliary:
            self.auxiliary_head = nn.Sequential(
                nn.Linear(image_features, 128),
                nn.ReLU(),
                nn.Dropout(0.2),
                nn.Linear(128, 2)  # Predict [NDVI, Height]
            )
    
    def forward(self, image, cat_data, cont_data, return_auxiliary=False):
        # Image features
        img_features = self.image_encoder(image)
        img_features = img_features.view(img_features.size(0), -1)  # Flatten
        
        # Tabular features
        cat_embeddings = [emb(cat_data[:, i]) for i, emb in enumerate(self.cat_embeddings)]
        cat_features = torch.cat(cat_embeddings, dim=1)
        tabular_input = torch.cat([cat_features, cont_data], dim=1)
        tab_features = self.tabular_encoder(tabular_input)
        
        # Combine features
        combined = torch.cat([img_features, tab_features], dim=1)
        
        # Main predictions
        biomass_pred = self.fusion_head(combined)
        
        # Auxiliary predictions (NDVI, Height from image only)
        if return_auxiliary and self.use_auxiliary:
            aux_pred = self.auxiliary_head(img_features)
            return biomass_pred, aux_pred
        
        return biomass_pred

# Create model
model = MultimodalBiomassPredictor(
    num_cat_features=2,
    cat_embedding_sizes=[5, 15],
    num_cont_features=4,
    num_outputs=5,
    image_model='resnet50',
    use_auxiliary=True
)
model = model.to(device)

print("Multimodal model created!")
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")

## 4. Custom Loss Function

Combine:
1. Main loss: Weighted MSE for biomass predictions (following competition weights)
2. Auxiliary loss: MSE for NDVI/Height predictions

In [ ]:
class CompetitionLoss(nn.Module):
    def __init__(self, use_auxiliary=True, aux_weight=0.1):
        super().__init__()
        self.use_auxiliary = use_auxiliary
        self.aux_weight = aux_weight
        
        # Competition weights for each target
        self.target_weights = torch.tensor([0.1, 0.1, 0.1, 0.2, 0.5]).to(device)
        # Order: Dry_Green, Dry_Dead, Dry_Clover, GDM, Dry_Total
        
        self.mse = nn.MSELoss(reduction='none')
    
    def forward(self, biomass_pred, biomass_true, aux_pred=None, aux_true=None):
        # Main biomass loss (weighted by competition metric)
        biomass_mse = self.mse(biomass_pred, biomass_true)
        weighted_biomass_loss = (biomass_mse * self.target_weights).mean()
        
        # Auxiliary loss (predict NDVI and Height)
        if self.use_auxiliary and aux_pred is not None and aux_true is not None:
            aux_loss = self.mse(aux_pred, aux_true).mean()
            total_loss = weighted_biomass_loss + self.aux_weight * aux_loss
            return total_loss, weighted_biomass_loss, aux_loss
        
        return weighted_biomass_loss, weighted_biomass_loss, torch.tensor(0.0).to(device)

criterion = CompetitionLoss(use_auxiliary=True, aux_weight=0.1)
print("Custom loss function created")

## 5. Training Setup

In [ ]:
# Optimizer
learning_rate = 3e-4
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)

# Scheduler - One Cycle (fast.ai style)
num_epochs = 10
steps_per_epoch = len(train_loader)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=learning_rate,
    epochs=num_epochs,
    steps_per_epoch=steps_per_epoch
)

print(f"Optimizer: AdamW (lr={learning_rate})")
print(f"Scheduler: OneCycleLR")
print(f"Epochs: {num_epochs}")

## 6. Training Loop

In [ ]:
def train_epoch(model, loader, criterion, optimizer, scheduler, device):
    model.train()
    total_loss = 0
    biomass_loss_sum = 0
    aux_loss_sum = 0
    
    for images, cat_data, cont_data, targets, ndvi_height in loader:
        images = images.to(device)
        cat_data = cat_data.to(device)
        cont_data = cont_data.to(device)
        targets = targets.to(device)
        ndvi_height = ndvi_height.to(device)
        
        optimizer.zero_grad()
        
        # Forward pass with auxiliary output
        biomass_pred, aux_pred = model(images, cat_data, cont_data, return_auxiliary=True)
        
        # Calculate loss
        loss, biomass_loss, aux_loss = criterion(biomass_pred, targets, aux_pred, ndvi_height)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        total_loss += loss.item() * images.size(0)
        biomass_loss_sum += biomass_loss.item() * images.size(0)
        aux_loss_sum += aux_loss.item() * images.size(0)
    
    return (total_loss / len(loader.dataset), 
            biomass_loss_sum / len(loader.dataset),
            aux_loss_sum / len(loader.dataset))

def validate_epoch(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for images, cat_data, cont_data, targets, ndvi_height in loader:
            images = images.to(device)
            cat_data = cat_data.to(device)
            cont_data = cont_data.to(device)
            targets = targets.to(device)
            
            # Forward pass (no auxiliary during validation)
            biomass_pred = model(images, cat_data, cont_data, return_auxiliary=False)
            
            # Calculate loss
            loss, _, _ = criterion(biomass_pred, targets)
            
            total_loss += loss.item() * images.size(0)
            all_preds.append(biomass_pred.cpu().numpy())
            all_targets.append(targets.cpu().numpy())
    
    all_preds = np.vstack(all_preds)
    all_targets = np.vstack(all_targets)
    
    return total_loss / len(loader.dataset), all_preds, all_targets

In [ ]:
# Training loop
best_val_loss = float('inf')
train_losses = []
val_losses = []
biomass_losses = []
aux_losses = []

print(f"Training for {num_epochs} epochs...\n")
print("="*80)

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    
    # Train
    train_loss, biomass_loss, aux_loss = train_epoch(
        model, train_loader, criterion, optimizer, scheduler, device
    )
    train_losses.append(train_loss)
    biomass_losses.append(biomass_loss)
    aux_losses.append(aux_loss)
    
    # Validate
    val_loss, val_preds, val_targets = validate_epoch(model, val_loader, criterion, device)
    val_losses.append(val_loss)
    
    print(f"  Train Loss: {train_loss:.4f} (Biomass: {biomass_loss:.4f}, Aux: {aux_loss:.4f})")
    print(f"  Val Loss:   {val_loss:.4f}")
    
    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_multimodal_model.pth')
        print(f"  ✓ Saved best model")
    
    print("-" * 80)

print("\nTraining complete!")

In [ ]:
# Plot training curves
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Loss curves
axes[0].plot(train_losses, label='Train Loss', marker='o')
axes[0].plot(val_losses, label='Val Loss', marker='o')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Progress')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Component losses
axes[1].plot(biomass_losses, label='Biomass Loss', marker='o')
axes[1].plot(aux_losses, label='Auxiliary Loss', marker='s')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].set_title('Loss Components')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Evaluate Performance

In [ ]:
# Load best model
model.load_state_dict(torch.load('best_multimodal_model.pth'))
print("Loaded best model\n")

# Get final validation predictions
val_loss, val_preds, val_targets = validate_epoch(model, val_loader, criterion, device)

# Calculate competition score
def calculate_competition_score(y_true, y_pred, target_cols):
    weights = {
        'Dry_Green_g': 0.1,
        'Dry_Dead_g': 0.1,
        'Dry_Clover_g': 0.1,
        'GDM_g': 0.2,
        'Dry_Total_g': 0.5
    }
    
    print("="*80)
    print("MULTIMODAL MODEL VALIDATION PERFORMANCE")
    print("="*80)
    
    r2_scores = {}
    total_score = 0
    
    for i, col in enumerate(target_cols):
        r2 = r2_score(y_true[:, i], y_pred[:, i])
        mae = mean_absolute_error(y_true[:, i], y_pred[:, i])
        r2_scores[col] = r2
        
        print(f"\n{col}:")
        print(f"  R² Score: {r2:.4f} (weight: {weights[col]})")
        print(f"  MAE: {mae:.2f}g")
        print(f"  Weighted contribution: {weights[col] * r2:.4f}")
        
        total_score += weights[col] * r2
    
    print("\n" + "="*80)
    print(f"COMPETITION SCORE (Weighted R²): {total_score:.4f}")
    print("="*80)
    
    return r2_scores, total_score

r2_scores, competition_score = calculate_competition_score(val_targets, val_preds, target_cols)

In [ ]:
# Visualize predictions
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for idx, col in enumerate(target_cols):
    ax = axes[idx]
    
    ax.scatter(val_targets[:, idx], val_preds[:, idx], alpha=0.5)
    
    min_val = min(val_targets[:, idx].min(), val_preds[:, idx].min())
    max_val = max(val_targets[:, idx].max(), val_preds[:, idx].max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect')
    
    ax.set_xlabel(f'Actual {col}')
    ax.set_ylabel(f'Predicted {col}')
    ax.set_title(f'{col}\nR² = {r2_scores[col]:.3f}')
    ax.legend()
    ax.grid(True, alpha=0.3)

fig.delaxes(axes[5])
plt.tight_layout()
plt.show()

## 8. Create Image-Only Model for Test Predictions

Since test data doesn't have tabular features, create a simplified version that uses only images.

In [ ]:
class ImageOnlyPredictor(nn.Module):
    """Simplified model for test time - uses only images"""
    def __init__(self, base_model):
        super().__init__()
        self.image_encoder = base_model.image_encoder
        
        # Direct path from image features to predictions
        image_features = 2048  # ResNet50
        self.head = nn.Sequential(
            nn.Linear(image_features, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.4),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.3),
            nn.Linear(256, 5)
        )
    
    def forward(self, image):
        img_features = self.image_encoder(image)
        img_features = img_features.view(img_features.size(0), -1)
        return self.head(img_features)

# Create image-only model and copy weights
image_only_model = ImageOnlyPredictor(model)
image_only_model = image_only_model.to(device)

print("Image-only model created for test predictions")

## 9. Generate Test Predictions

In [ ]:
# Load test data
test_df = pd.read_csv('competition/test.csv')
test_images = test_df['image_path'].unique()

print(f"Test images: {len(test_images)}")

# Create simple test dataset (images only)
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

class SimpleTestDataset(Dataset):
    def __init__(self, image_paths, root='competition', transform=None):
        self.image_paths = image_paths
        self.root = root
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = f"{self.root}/{self.image_paths[idx]}"
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

test_dataset = SimpleTestDataset(test_images, transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=0)

# Generate predictions
image_only_model.eval()
test_predictions = []

with torch.no_grad():
    for images in test_loader:
        images = images.to(device)
        outputs = image_only_model(images)
        test_predictions.append(outputs.cpu().numpy())

test_predictions = np.vstack(test_predictions)
print(f"\nTest predictions shape: {test_predictions.shape}")

In [ ]:
# Create submission
pred_dict = {}
for idx, img_path in enumerate(test_images):
    for col_idx, col in enumerate(target_cols):
        pred_dict[img_path + '__' + col] = test_predictions[idx, col_idx]

submission = test_df.copy()
submission['target'] = submission.apply(
    lambda row: pred_dict.get(row['image_path'] + '__' + row['target_name'], 0.0),
    axis=1
)

# Clip negative predictions
submission['target'] = submission['target'].clip(lower=0)

# Save
submission[['sample_id', 'target']].to_csv('submission_multimodal.csv', index=False)
print("\nSubmission saved to submission_multimodal.csv")
print(submission[['sample_id', 'target']].head(10))

## Summary

### Multimodal Architecture:
- **Image branch**: ResNet50 pre-trained features
- **Tabular branch**: Embeddings + dense layers for NDVI, Height, State, Species, Date
- **Late fusion**: Concatenate and predict biomass
- **Auxiliary learning**: Model learns to predict NDVI/Height from images during training

### Key Improvements:
1. Uses ALL available data during training (images + tabular)
2. Auxiliary task helps model learn relevant visual features
3. Weighted loss function aligned with competition metric
4. One-cycle learning rate schedule
5. Better data augmentation

### Performance:
Check the validation R² scores and competition score above to see improvement over image-only baseline!

### Next Steps:
- Try different architectures (EfficientNet, Vision Transformer)
- Experiment with auxiliary task weighting
- Ensemble multiple models
- Cross-validation for more robust evaluation